In [ ]:
from flask import Flask, render_template, request, jsonify, redirect, url_for, session, send_from_directory
from pymongo import MongoClient
from werkzeug.security import generate_password_hash, check_password_hash
import os
import uuid
import logging
import cv2
import numpy as np
from tensorflow.keras.models import load_model

app = Flask(__name__)
app.secret_key = 'your-secret-key'  # Replace with a secure key in production

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# MongoDB connection
try:
    client = MongoClient('mongodb://localhost:27017')
    db = client['fire_detection']
    users = db['users']
    logger.info("MongoDB connected successfully")
except Exception as e:
    logger.error(f"Failed to connect to MongoDB: {str(e)}")
    raise

# Directories
UPLOAD_FOLDER = 'uploads'
EXTRACTED_FOLDER = 'extracted_clips'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(EXTRACTED_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Model path
MODEL_PATH = r"C:\Users\PMLS\Documents\ai fire detection frontend\fire_detection_model_image.h5"

# Load the trained model
try:
    model = load_model(MODEL_PATH)
    logger.info("Model loaded successfully")
except Exception as e:
    logger.error(f"Failed to load model: {str(e)}")
    raise

# Store progress (simple in-memory storage; use Redis/database in production)
progress_data = {}

def extract_fire_clips(
    video_path,
    output_dir,
    progress_id=None,
    frame_skip=5,
    min_clip_length=5,   # in seconds
    frame_threshold=0.3, # threshold to consider fire
    vote_window=10,      # number of frames in voting window
    vote_majority=3      # minimum fire votes in window to consider fire
):
    """
    Extract fire clips from a video using dynamic frame skipping based on model predictions.
    """
    try:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Created output directory: {output_dir}")

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logger.error(f"Failed to open video: {video_path}")
            raise ValueError("Invalid or corrupted video file")
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        logger.info(f"Processing video: {video_path}, Total frames: {total_frames}, FPS: {fps}")

        fire_predictions = []
        frame_num = 0

        print("[INFO] Starting frame analysis with dynamic frame skipping...")

        while frame_num < total_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()

            if not ret:
                logger.warning(f"Failed to read frame {frame_num}")
                break

            frame_resized = cv2.resize(frame, (299, 299))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            frame_norm = frame_rgb.astype(np.float32) / 255.0

            try:
                fire_prob = model.predict(np.expand_dims(frame_norm, axis=0), verbose=0)[0][0]
                print(f"Frame {frame_num}: Fire probability = {fire_prob:.2f}")
            except Exception as e:
                logger.error(f"Prediction error at frame {frame_num}: {str(e)}")
                raise

            fire_predictions.append((frame_num, fire_prob))

            # Update progress
            if progress_id and progress_id in progress_data:
                progress = (frame_num / total_frames) * 100
                progress_data[progress_id]['progress'] = progress
                logger.debug(f"Progress for {progress_id}: {progress:.2f}%")

            # Dynamic frame skipping
            if fire_prob > frame_threshold:
                frame_num += 1
            else:
                frame_num += frame_skip

        cap.release()

        # Apply voting
        fire_frames = []
        for i in range(len(fire_predictions) - vote_window + 1):
            window = fire_predictions[i:i + vote_window]
            fire_votes = sum(1 for (_, prob) in window if prob > frame_threshold)

            if fire_votes >= vote_majority:
                mid_frame = window[vote_window // 2][0]
                fire_frames.append(mid_frame)

        fire_frames = sorted(set(fire_frames))

        # Group frames into clips
        clips = []
        current_clip = []

        for i, frame_num in enumerate(fire_frames):
            if not current_clip:
                current_clip.append(frame_num)
            elif frame_num == current_clip[-1] + 1:
                current_clip.append(frame_num)
            else:
                if len(current_clip) / fps >= min_clip_length:
                    clips.append(current_clip)
                current_clip = [frame_num]

        if len(current_clip) / fps >= min_clip_length:
            clips.append(current_clip)

        print(f"[INFO] Total fire clips found: {len(clips)}")
        logger.info(f"Total fire clips found: {len(clips)}")

        # Save clips
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logger.error(f"Failed to reopen video for saving clips: {video_path}")
            raise ValueError("Failed to reopen video file")

        for idx, clip in enumerate(clips):
            output_clip_path = os.path.join(output_dir, f"fire_clip_{idx + 1}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_clip_path, fourcc, fps, (299, 299))

            if not out.isOpened():
                logger.error(f"Failed to create output video: {output_clip_path}")
                raise IOError("Failed to create output video")

            for frame_num in clip:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                ret, frame = cap.read()
                if ret:
                    frame_resized = cv2.resize(frame, (299, 299))
                    out.write(frame_resized)
                else:
                    logger.warning(f"Failed to read frame {frame_num} for clip {idx + 1}")

            out.release()
            print(f"[SAVED] {output_clip_path}")
            logger.info(f"Saved clip: {output_clip_path}")

        cap.release()
        print(f"[DONE] Extracted {len(clips)} fire clips to '{output_dir}'")
        logger.info(f"Extracted {len(clips)} fire clips to '{output_dir}'")

    except Exception as e:
        logger.error(f"Error in extract_fire_clips: {str(e)}", exc_info=True)
        raise

# Routes
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        user = users.find_one({'email': email})
        if user and check_password_hash(user['password'], password):
            session['user_id'] = str(user['_id'])
            return redirect(url_for('upload'))
        return render_template('login.html', error='Invalid credentials')
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        if users.find_one({'email': email}):
            return render_template('register.html', error='Email already exists')
        hashed_password = generate_password_hash(password)
        users.insert_one({
            'username': username,
            'email': email,
            'password': hashed_password
        })
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    if request.method == 'POST':
        try:
            if 'video' not in request.files:
                logger.error("No video file provided in request")
                return jsonify({'error': 'No video file provided'}), 400
            file = request.files['video']
            if file.filename == '':
                logger.error("No selected file")
                return jsonify({'error': 'No selected file'}), 400
            filename = f"{uuid.uuid4()}_{file.filename}"
            video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(video_path)
            logger.info(f"Video saved to {video_path}")
            
            # Store processing session
            processing_id = str(uuid.uuid4())
            session['processing_id'] = processing_id
            progress_data[processing_id] = {'progress': 0, 'done': False}
            logger.debug(f"Started processing with ID: {processing_id}")
            
            # Process video synchronously
            try:
                extract_fire_clips(
                    video_path=video_path,
                    output_dir=EXTRACTED_FOLDER,
                    progress_id=processing_id,
                    frame_skip=5,
                    min_clip_length=5,
                    frame_threshold=0.3,
                    vote_window=10,
                    vote_majority=3
                )
                progress_data[processing_id]['done'] = True
                logger.info("Video processing completed")
            except Exception as e:
                logger.error(f"Failed to process video with extract_fire_clips: {str(e)}", exc_info=True)
                raise ValueError(f"Video processing failed: {str(e)}")
            
            return jsonify({'message': 'Processing complete', 'redirect': url_for('results')})
        except Exception as e:
            logger.error(f"Error processing video: {str(e)}", exc_info=True)
            return jsonify({'error': f"Server error: {str(e)}"}), 500
    return render_template('upload.html')

@app.route('/progress')
def get_progress():
    try:
        processing_id = session.get('processing_id')
        if processing_id and processing_id in progress_data:
            return jsonify(progress_data[processing_id])
        logger.warning("No progress data found for session")
        return jsonify({'progress': 0, 'done': False})
    except Exception as e:
        logger.error(f"Error fetching progress: {str(e)}")
        return jsonify({'error': f"Server error: {str(e)}"}), 500

@app.route('/results')
def results():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    try:
        clips = [
            {'url': f"/extracted_clips/{file}"}
            for file in os.listdir(EXTRACTED_FOLDER)
            if file.endswith('.mp4')
        ]
        logger.info(f"Found {len(clips)} clips in results")
        return render_template('results.html', clips=clips)
    except Exception as e:
        logger.error(f"Error loading results: {str(e)}")
        return render_template('results.html', clips=[], error=str(e))

@app.route('/logout')
def logout():
    session.pop('user_id', None)
    session.pop('processing_id', None)
    return redirect(url_for('index'))

# Serve extracted clips
@app.route('/extracted_clips/<filename>')
def serve_clip(filename):
    try:
        return send_from_directory(EXTRACTED_FOLDER, filename)
    except Exception as e:
        logger.error(f"Error serving clip {filename}: {str(e)}")
        return jsonify({'error': f"File not found: {str(e)}"}), 404

if __name__ == '__main__':
    app.run()

2025-05-15 09:14:18,054 - DEBUG - {"clientId": {"$oid": "68256a199a6d518de2c258a7"}, "message": "Connection pool created", "serverHost": "localhost", "serverPort": 27017}
2025-05-15 09:14:18,064 - INFO - MongoDB connected successfully
2025-05-15 09:14:18,119 - DEBUG - Creating converter from 3 to 5
2025-05-15 09:14:18,248 - DEBUG - {"clientId": {"$oid": "68256a199a6d518de2c258a7"}, "message": "Connection pool ready", "serverHost": "localhost", "serverPort": 27017}
2025-05-15 09:14:24,235 - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-05-15 09:14:24,263 - INFO - Model loaded successfully


 * Serving Flask app '__main__'
 * Debug mode: off


2025-05-15 09:14:24,328 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-05-15 09:14:24,329 - INFO - Press CTRL+C to quit
2025-05-15 09:14:28,532 - INFO - 127.0.0.1 - - [15/May/2025 09:14:28] "GET / HTTP/1.1" 200 -
2025-05-15 09:14:28,784 - INFO - 127.0.0.1 - - [15/May/2025 09:14:28] "GET /static/css/index.css HTTP/1.1" 304 -
2025-05-15 09:14:29,656 - INFO - 127.0.0.1 - - [15/May/2025 09:14:29] "GET /favicon.ico HTTP/1.1" 404 -
2025-05-15 09:14:37,216 - INFO - 127.0.0.1 - - [15/May/2025 09:14:37] "GET /login HTTP/1.1" 200 -
2025-05-15 09:14:37,377 - INFO - 127.0.0.1 - - [15/May/2025 09:14:37] "GET /static/css/login.css HTTP/1.1" 304 -
2025-05-15 09:14:40,118 - DEBUG - {"message": "Server selection started", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 68256a199a6d518de2c258a7, topology_type: Single, servers: [<ServerDe

[INFO] Starting frame analysis with dynamic frame skipping...


2025-05-15 09:15:47,691 - DEBUG - Progress for a378e1ea-fd90-4979-897f-39891ad909be: 0.00%


...

Frame 100: Fire probability = 0.09


2025-05-15 09:17:12,847 - DEBUG - Progress for b4146307-021d-4113-9842-6bbeb7857ba7: 94.59%


Frame 105: Fire probability = 0.05


2025-05-15 09:17:14,514 - DEBUG - Progress for b4146307-021d-4113-9842-6bbeb7857ba7: 99.10%
2025-05-15 09:17:14,537 - INFO - Total fire clips found: 0
2025-05-15 09:17:14,626 - INFO - Extracted 0 fire clips to 'extracted_clips'
2025-05-15 09:17:14,630 - INFO - Video processing completed
2025-05-15 09:17:14,637 - INFO - 127.0.0.1 - - [15/May/2025 09:17:14] "POST /upload HTTP/1.1" 200 -


Frame 110: Fire probability = 0.07
[INFO] Total fire clips found: 0
[DONE] Extracted 0 fire clips to 'extracted_clips'


Session

In [1]:
from flask import Flask, render_template, request, jsonify, redirect, url_for, session, send_from_directory
from pymongo import MongoClient
from werkzeug.security import generate_password_hash, check_password_hash
import os
import uuid
import logging
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from datetime import timedelta

app = Flask(__name__)
app.secret_key = 'your-secret-key'  # Replace with a secure key in production
app.permanent_session_lifetime = timedelta(days=7)  # Sessions last 7 days

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# MongoDB connection
try:
    client = MongoClient('mongodb://localhost:27017')
    db = client['fire_detection']
    users = db['users']
    logger.info("MongoDB connected successfully")
except Exception as e:
    logger.error(f"Failed to connect to MongoDB: {str(e)}")
    raise

# Directories
UPLOAD_FOLDER = 'uploads'
EXTRACTED_FOLDER = 'extracted_clips'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(EXTRACTED_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Model path
MODEL_PATH = r"C:\Users\PMLS\Documents\ai fire detection frontend\fire_detection_model_image.h5"

# Load the trained model
try:
    model = load_model(MODEL_PATH)
    # Optional: Compile model to suppress warning (unnecessary for inference)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    logger.info("Model loaded successfully")
except Exception as e:
    logger.error(f"Failed to load model: {str(e)}")
    raise

# Store progress (simple in-memory storage; use Redis/database in production)
progress_data = {}

def extract_fire_clips(
    video_path,
    output_dir,
    progress_id=None,
    frame_skip=5,
    min_clip_length=5,   # in seconds
    frame_threshold=0.3, # threshold to consider fire
    vote_window=10,      # number of frames in voting window
    vote_majority=3      # minimum fire votes in window to consider fire
):
    """
    Extract fire clips from a video using dynamic frame skipping based on model predictions.
    """
    try:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Created output directory: {output_dir}")

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logger.error(f"Failed to open video: {video_path}")
            raise ValueError("Invalid or corrupted video file")
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        logger.info(f"Processing video: {video_path}, Total frames: {total_frames}, FPS: {fps}")

        fire_predictions = []
        frame_num = 0

        print("[INFO] Starting frame analysis with dynamic frame skipping...")
        logger.debug(f"Starting frame analysis for progress_id: {progress_id}")

        while frame_num < total_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()

            if not ret:
                logger.warning(f"Failed to read frame {frame_num}")
                break

            frame_resized = cv2.resize(frame, (299, 299))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            frame_norm = frame_rgb.astype(np.float32) / 255.0

            try:
                fire_prob = model.predict(np.expand_dims(frame_norm, axis=0), verbose=0)[0][0]
                print(f"Frame {frame_num}: Fire probability = {fire_prob:.2f}")
                logger.debug(f"Frame {frame_num}: Fire probability = {fire_prob:.2f}")
            except Exception as e:
                logger.error(f"Prediction error at frame {frame_num}: {str(e)}")
                raise

            fire_predictions.append((frame_num, fire_prob))

            # Update progress
            if progress_id and progress_id in progress_data:
                progress = (frame_num / total_frames) * 100
                progress_data[progress_id]['progress'] = progress
                logger.debug(f"Progress for {progress_id}: {progress:.2f}%")
            else:
                logger.warning(f"Progress not updated: progress_id {progress_id} not in progress_data")

            # Dynamic frame skipping
            if fire_prob > frame_threshold:
                frame_num += 1
            else:
                frame_num += frame_skip

        cap.release()

        # Apply voting
        fire_frames = []
        for i in range(len(fire_predictions) - vote_window + 1):
            window = fire_predictions[i:i + vote_window]
            fire_votes = sum(1 for (_, prob) in window if prob > frame_threshold)

            if fire_votes >= vote_majority:
                mid_frame = window[vote_window // 2][0]
                fire_frames.append(mid_frame)

        fire_frames = sorted(set(fire_frames))

        # Group frames into clips
        clips = []
        current_clip = []

        for i, frame_num in enumerate(fire_frames):
            if not current_clip:
                current_clip.append(frame_num)
            elif frame_num == current_clip[-1] + 1:
                current_clip.append(frame_num)
            else:
                if len(current_clip) / fps >= min_clip_length:
                    clips.append(current_clip)
                current_clip = [frame_num]

        if len(current_clip) / fps >= min_clip_length:
            clips.append(current_clip)

        print(f"[INFO] Total fire clips found: {len(clips)}")
        logger.info(f"Total fire clips found: {len(clips)}")

        # Save clips
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logger.error(f"Failed to reopen video for saving clips: {video_path}")
            raise ValueError("Failed to reopen video file")

        for idx, clip in enumerate(clips):
            output_clip_path = os.path.join(output_dir, f"fire_clip_{idx + 1}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_clip_path, fourcc, fps, (299, 299))

            if not out.isOpened():
                logger.error(f"Failed to create output video: {output_clip_path}")
                raise IOError("Failed to create output video")

            for frame_num in clip:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                ret, frame = cap.read()
                if ret:
                    frame_resized = cv2.resize(frame, (299, 299))
                    out.write(frame_resized)
                else:
                    logger.warning(f"Failed to read frame {frame_num} for clip {idx + 1}")

            out.release()
            print(f"[SAVED] {output_clip_path}")
            logger.info(f"Saved clip: {output_clip_path}")

        cap.release()
        print(f"[DONE] Extracted {len(clips)} fire clips to '{output_dir}'")
        logger.info(f"Extracted {len(clips)} fire clips to '{output_dir}'")

    except Exception as e:
        logger.error(f"Error in extract_fire_clips: {str(e)}", exc_info=True)
        raise

# Routes
@app.route('/')
def index():
    if 'user_id' in session:
        logger.debug("User already logged in, redirecting to upload")
        return redirect(url_for('upload'))
    return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if 'user_id' in session:
        logger.debug("User already logged in, redirecting to upload")
        return redirect(url_for('upload'))
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        user = users.find_one({'email': email})
        if user and check_password_hash(user['password'], password):
            session['user_id'] = str(user['_id'])
            session.permanent = True  # Make session persistent
            logger.info(f"User {email} logged in successfully")
            return redirect(url_for('upload'))
        return render_template('login.html', error='Invalid credentials')
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if 'user_id' in session:
        logger.debug("User already logged in, redirecting to upload")
        return redirect(url_for('upload'))
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        if users.find_one({'email': email}):
            return render_template('register.html', error='Email already exists')
        hashed_password = generate_password_hash(password)
        users.insert_one({
            'username': username,
            'email': email,
            'password': hashed_password
        })
        logger.info(f"User {email} registered successfully")
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if 'user_id' not in session:
        logger.debug("No user session, redirecting to login")
        return redirect(url_for('login'))
    if request.method == 'POST':
        try:
            if 'video' not in request.files:
                logger.error("No video file provided in request")
                return jsonify({'error': 'No video file provided'}), 400
            file = request.files['video']
            if file.filename == '':
                logger.error("No selected file")
                return jsonify({'error': 'No selected file'}), 400
            
            # Check if processing is already in progress
            if session.get('processing_id') and session['processing_id'] in progress_data and not progress_data[session['processing_id']]['done']:
                logger.warning("Processing already in progress for this session")
                return jsonify({'error': 'Processing already in progress'}), 400
            
            filename = f"{uuid.uuid4()}_{file.filename}"
            video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(video_path)
            logger.info(f"Video saved to {video_path}")
            
            # Store processing session
            processing_id = str(uuid.uuid4())
            session['processing_id'] = processing_id
            progress_data[processing_id] = {'progress': 0, 'done': False}
            logger.debug(f"Started processing with ID: {processing_id}")
            
            # Process video synchronously
            try:
                extract_fire_clips(
                    video_path=video_path,
                    output_dir=EXTRACTED_FOLDER,
                    progress_id=processing_id,
                    frame_skip=5,
                    min_clip_length=5,
                    frame_threshold=0.3,
                    vote_window=10,
                    vote_majority=3
                )
                progress_data[processing_id]['done'] = True
                logger.info("Video processing completed")
            except Exception as e:
                logger.error(f"Failed to process video with extract_fire_clips: {str(e)}", exc_info=True)
                raise ValueError(f"Video processing failed: {str(e)}")
            
            return jsonify({'message': 'Processing complete', 'redirect': url_for('results')})
        except Exception as e:
            logger.error(f"Error processing video: {str(e)}", exc_info=True)
            return jsonify({'error': f"Server error: {str(e)}"}), 500
    return render_template('upload.html')

@app.route('/progress')
def get_progress():
    try:
        processing_id = session.get('processing_id')
        logger.debug(f"Fetching progress for processing_id: {processing_id}")
        if processing_id and processing_id in progress_data:
            logger.debug(f"Progress data: {progress_data[processing_id]}")
            return jsonify(progress_data[processing_id])
        logger.warning("No progress data found for session")
        return jsonify({'progress': 0, 'done': False})
    except Exception as e:
        logger.error(f"Error fetching progress: {str(e)}")
        return jsonify({'error': f"Server error: {str(e)}"}), 500

@app.route('/results')
def results():
    if 'user_id' not in session:
        logger.debug("No user session, redirecting to login")
        return redirect(url_for('login'))
    try:
        clips = [
            {'url': f"/extracted_clips/{file}"}
            for file in os.listdir(EXTRACTED_FOLDER)
            if file.endswith('.mp4')
        ]
        logger.info(f"Found {len(clips)} clips in results")
        return render_template('results.html', clips=clips)
    except Exception as e:
        logger.error(f"Error loading results: {str(e)}")
        return render_template('results.html', clips=[], error=str(e))

@app.route('/logout')
def logout():
    session.pop('user_id', None)
    session.pop('processing_id', None)
    logger.info("User logged out")
    return render_template('logout.html')

# Serve extracted clips
@app.route('/extracted_clips/<filename>')
def serve_clip(filename):
    try:
        return send_from_directory(EXTRACTED_FOLDER, filename)
    except Exception as e:
        logger.error(f"Error serving clip {filename}: {str(e)}")
        return jsonify({'error': f"File not found: {str(e)}"}), 404

if __name__ == '__main__':
    app.run()

2025-05-24 19:11:59,994 - DEBUG - {"clientId": {"$oid": "6831d3af7800c355f733fec0"}, "message": "Connection pool created", "serverHost": "localhost", "serverPort": 27017}
2025-05-24 19:11:59,997 - INFO - MongoDB connected successfully
2025-05-24 19:12:00,046 - DEBUG - Creating converter from 3 to 5
2025-05-24 19:12:00,137 - DEBUG - {"clientId": {"$oid": "6831d3af7800c355f733fec0"}, "message": "Connection pool ready", "serverHost": "localhost", "serverPort": 27017}
2025-05-24 19:12:03,462 - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-05-24 19:12:03,476 - INFO - Model loaded successfully


 * Serving Flask app '__main__'
 * Debug mode: off


2025-05-24 19:12:03,511 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-05-24 19:12:03,512 - INFO - Press CTRL+C to quit
2025-05-24 19:12:05,759 - INFO - 127.0.0.1 - - [24/May/2025 19:12:05] "GET / HTTP/1.1" 200 -
2025-05-24 19:12:06,195 - INFO - 127.0.0.1 - - [24/May/2025 19:12:06] "GET /static/css/index.css HTTP/1.1" 200 -
2025-05-24 19:12:07,141 - INFO - 127.0.0.1 - - [24/May/2025 19:12:07] "GET /favicon.ico HTTP/1.1" 404 -
2025-05-24 19:12:11,210 - INFO - 127.0.0.1 - - [24/May/2025 19:12:11] "GET /login HTTP/1.1" 200 -
2025-05-24 19:12:11,311 - INFO - 127.0.0.1 - - [24/May/2025 19:12:11] "GET /static/css/login.css HTTP/1.1" 200 -
2025-05-24 19:12:22,291 - DEBUG - {"message": "Server selection started", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6831d3af7800c355f733fec0, topology_type: Single, servers: [<ServerDe

TASHFEEN CODE 

In [2]:
from flask import Flask, render_template, request, jsonify, redirect, url_for, session, send_from_directory
from pymongo import MongoClient
from werkzeug.security import generate_password_hash, check_password_hash
import os
import uuid
import logging
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from datetime import timedelta

app = Flask(__name__)
app.secret_key = 'your-secret-key'  # Replace with a secure key in production
app.permanent_session_lifetime = timedelta(days=7)  # Sessions last 7 days

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# MongoDB connection
try:
    client = MongoClient('mongodb://localhost:27017')
    db = client['fire_detection']
    users = db['users']
    logger.info("MongoDB connected successfully")
except Exception as e:
    logger.error(f"Failed to connect to MongoDB: {str(e)}")
    raise

# Directories
UPLOAD_FOLDER = 'uploads'
EXTRACTED_FOLDER = 'extracted_clips'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(EXTRACTED_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Model path
MODEL_PATH = r"C:\Users\PMLS\Documents\ai fire detection frontend\fire_detection_model_image.h5"

# Load the trained model
try:
    model = load_model(MODEL_PATH)
    # Optional: Compile model to suppress warning (unnecessary for inference)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    logger.info("Model loaded successfully")
except Exception as e:
    logger.error(f"Failed to load model: {str(e)}")
    raise

# Store progress (simple in-memory storage; use Redis/database in production)
progress_data = {}

def extract_fire_clips(
    video_path,
    output_dir,
    progress_id=None,
    initial_frame_skip=30,
    frame_threshold=0.3,
    dynamic_skip_factor=5
):
    """
    Extract frames with fire from video and combine into one output video clip.
    Dynamically reduce frame skipping when fire is detected.
    """
    try:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Created output directory: {output_dir}")

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logger.error(f"Failed to open video: {video_path}")
            raise ValueError("Invalid or corrupted video file")

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        logger.info(f"Processing video: {video_path}, Total frames: {total_frames}, FPS: {fps}")

        frame_num = 0
        frame_skip = initial_frame_skip
        fire_detected_frames = []

        print("[INFO] Starting frame analysis with dynamic skip...")

        while frame_num < total_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()

            if not ret:
                logger.warning(f"Failed to read frame {frame_num}")
                break

            frame_resized = cv2.resize(frame, (299, 299))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            frame_norm = frame_rgb.astype(np.float32) / 255.0

            try:
                fire_prob = model.predict(np.expand_dims(frame_norm, axis=0), verbose=0)[0][0]
                print(f"Frame {frame_num}: Fire probability = {fire_prob:.2f}")
                logger.debug(f"Frame {frame_num}: Fire probability = {fire_prob:.2f}")
            except Exception as e:
                logger.error(f"Prediction error at frame {frame_num}: {str(e)}")
                raise

            if fire_prob > frame_threshold:
                fire_detected_frames.append(frame_resized)
                frame_skip = max(1, frame_skip // dynamic_skip_factor)
            else:
                frame_skip = initial_frame_skip

            # Update progress
            if progress_id and progress_id in progress_data:
                progress = (frame_num / total_frames) * 100
                progress_data[progress_id]['progress'] = progress
                logger.debug(f"Progress for {progress_id}: {progress:.2f}%")
            else:
                logger.warning(f"Progress not updated: progress_id {progress_id} not in progress_data")

            frame_num += frame_skip

        cap.release()

        # Save all fire frames into a single video
        if fire_detected_frames:
            output_clip_path = os.path.join(output_dir, f"fire_clip_combined.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_clip_path, fourcc, fps, (299, 299))

            if not out.isOpened():
                logger.error(f"Failed to create output video: {output_clip_path}")
                raise IOError("Failed to create output video")

            for frame in fire_detected_frames:
                out.write(frame)

            out.release()
            print(f"[SAVED] Combined fire clip: {output_clip_path}")
            logger.info(f"Saved combined fire clip: {output_clip_path}")
        else:
            print("[INFO] No fire detected in video.")
            logger.info("No fire frames detected to save.")

    except Exception as e:
        logger.error(f"Error in extract_fire_clips: {str(e)}", exc_info=True)
        raise


# Routes
@app.route('/')
def index():
    if 'user_id' in session:
        logger.debug("User already logged in, redirecting to upload")
        return redirect(url_for('upload'))
    return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if 'user_id' in session:
        logger.debug("User already logged in, redirecting to upload")
        return redirect(url_for('upload'))
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        user = users.find_one({'email': email})
        if user and check_password_hash(user['password'], password):
            session['user_id'] = str(user['_id'])
            session.permanent = True  # Make session persistent
            logger.info(f"User {email} logged in successfully")
            return redirect(url_for('upload'))
        return render_template('login.html', error='Invalid credentials')
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if 'user_id' in session:
        logger.debug("User already logged in, redirecting to upload")
        return redirect(url_for('upload'))
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        if users.find_one({'email': email}):
            return render_template('register.html', error='Email already exists')
        hashed_password = generate_password_hash(password)
        users.insert_one({
            'username': username,
            'email': email,
            'password': hashed_password
        })
        logger.info(f"User {email} registered successfully")
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if 'user_id' not in session:
        logger.debug("No user session, redirecting to login")
        return redirect(url_for('login'))
    if request.method == 'POST':
        try:
            if 'video' not in request.files:
                logger.error("No video file provided in request")
                return jsonify({'error': 'No video file provided'}), 400
            file = request.files['video']
            if file.filename == '':
                logger.error("No selected file")
                return jsonify({'error': 'No selected file'}), 400
            
            # Check if processing is already in progress
            if session.get('processing_id') and session['processing_id'] in progress_data and not progress_data[session['processing_id']]['done']:
                logger.warning("Processing already in progress for this session")
                return jsonify({'error': 'Processing already in progress'}), 400
            
            filename = f"{uuid.uuid4()}_{file.filename}"
            video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(video_path)
            logger.info(f"Video saved to {video_path}")
            
            # Store processing session
            processing_id = str(uuid.uuid4())
            session['processing_id'] = processing_id
            progress_data[processing_id] = {'progress': 0, 'done': False}
            logger.debug(f"Started processing with ID: {processing_id}")
            
            # Process video synchronously
            try:
                extract_fire_clips(
                    video_path=video_path,
                    output_dir=EXTRACTED_FOLDER,
                    progress_id=processing_id,
                    frame_skip=5,
                    min_clip_length=5,
                    frame_threshold=0.3,
                    vote_window=10,
                    vote_majority=3
                )
                progress_data[processing_id]['done'] = True
                logger.info("Video processing completed")
            except Exception as e:
                logger.error(f"Failed to process video with extract_fire_clips: {str(e)}", exc_info=True)
                raise ValueError(f"Video processing failed: {str(e)}")
            
            return jsonify({'message': 'Processing complete', 'redirect': url_for('results')})
        except Exception as e:
            logger.error(f"Error processing video: {str(e)}", exc_info=True)
            return jsonify({'error': f"Server error: {str(e)}"}), 500
    return render_template('upload.html')

@app.route('/progress')
def get_progress():
    try:
        processing_id = session.get('processing_id')
        logger.debug(f"Fetching progress for processing_id: {processing_id}")
        if processing_id and processing_id in progress_data:
            logger.debug(f"Progress data: {progress_data[processing_id]}")
            return jsonify(progress_data[processing_id])
        logger.warning("No progress data found for session")
        return jsonify({'progress': 0, 'done': False})
    except Exception as e:
        logger.error(f"Error fetching progress: {str(e)}")
        return jsonify({'error': f"Server error: {str(e)}"}), 500

@app.route('/results')
def results():
    if 'user_id' not in session:
        logger.debug("No user session, redirecting to login")
        return redirect(url_for('login'))
    try:
        clips = [
            {'url': f"/extracted_clips/{file}"}
            for file in os.listdir(EXTRACTED_FOLDER)
            if file.endswith('.mp4')
        ]
        logger.info(f"Found {len(clips)} clips in results")
        return render_template('results.html', clips=clips)
    except Exception as e:
        logger.error(f"Error loading results: {str(e)}")
        return render_template('results.html', clips=[], error=str(e))

@app.route('/logout')
def logout():
    session.pop('user_id', None)
    session.pop('processing_id', None)
    logger.info("User logged out")
    return render_template('logout.html')

# Serve extracted clips
@app.route('/extracted_clips/<filename>')
def serve_clip(filename):
    try:
        return send_from_directory(EXTRACTED_FOLDER, filename)
    except Exception as e:
        logger.error(f"Error serving clip {filename}: {str(e)}")
        return jsonify({'error': f"File not found: {str(e)}"}), 404

if __name__ == '__main__':
    app.run()

2025-05-24 19:14:55,855 - DEBUG - {"clientId": {"$oid": "6831d45f7800c355f733fec1"}, "message": "Connection pool created", "serverHost": "localhost", "serverPort": 27017}
2025-05-24 19:14:55,862 - INFO - MongoDB connected successfully
2025-05-24 19:14:55,862 - DEBUG - {"clientId": {"$oid": "6831d45f7800c355f733fec1"}, "message": "Connection pool ready", "serverHost": "localhost", "serverPort": 27017}
2025-05-24 19:14:59,260 - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-05-24 19:14:59,269 - INFO - Model loaded successfully


 * Serving Flask app '__main__'
 * Debug mode: off


2025-05-24 19:14:59,282 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-05-24 19:14:59,283 - INFO - Press CTRL+C to quit
2025-05-24 19:15:01,599 - DEBUG - User already logged in, redirecting to upload
2025-05-24 19:15:01,605 - INFO - 127.0.0.1 - - [24/May/2025 19:15:01] "GET / HTTP/1.1" 302 -
2025-05-24 19:15:01,653 - INFO - 127.0.0.1 - - [24/May/2025 19:15:01] "GET /upload HTTP/1.1" 200 -
2025-05-24 19:15:01,804 - INFO - 127.0.0.1 - - [24/May/2025 19:15:01] "GET /static/css/upload.css HTTP/1.1" 304 -
2025-05-24 19:15:01,823 - INFO - 127.0.0.1 - - [24/May/2025 19:15:01] "GET /static/js/upload.js HTTP/1.1" 304 -
